In [1]:
import pandas as pd
import matplotlib.pyplot as plt 
import numpy as np
import seaborn as sns
import plotly.express as px
import plotly.graph_objects as go
import plotly.figure_factory as ff
from datetime import datetime

In [2]:
# Load and prepare the dataset
df = pd.read_csv('Anti_Fraud_Centre_Data.csv')

# Convert date column to datetime
df['Date'] = pd.to_datetime(df['Date Received / Date reue'])

# Clean dollar loss data
df['Dollar Loss Clean'] = df['Dollar Loss /pertes financires'].astype(str).str.replace('$', '')
df['Dollar Loss Clean'] = df['Dollar Loss Clean'].str.replace(',', '')
df['Dollar Loss Clean'] = pd.to_numeric(df['Dollar Loss Clean'], errors='coerce')

# Extract year and month
df['Year'] = df['Date'].dt.year
df['Month'] = df['Date'].dt.month
df['YearMonth'] = df['Date'].dt.to_period('M')

In [3]:
# 1. Number of fraud cases by year/month (Line Chart)
monthly_cases = df.groupby('YearMonth').size().reset_index()
monthly_cases.columns = ['YearMonth', 'Count']
monthly_cases['YearMonth'] = monthly_cases['YearMonth'].astype(str)

fig1 = go.Figure()

fig1.add_trace(go.Scatter(
    x=monthly_cases['YearMonth'],
    y=monthly_cases['Count'],
    mode='lines+markers',
    name='Number of Cases',
    line=dict(color='rgb(67, 147, 195)', width=2),
    marker=dict(size=6)
))

fig1.update_layout(
    title={
        'text': 'Number of Fraud Cases Over Time',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Time Period',
    yaxis_title='Number of Cases',
    template='plotly_white',
    showlegend=False,
    plot_bgcolor='white',
    xaxis=dict(tickangle=45)
)

fig1.show()

In [4]:
# 2. Dollar loss trends over time (Area Chart)
monthly_loss = df.groupby('YearMonth').agg({
    'Dollar Loss Clean': 'sum',
    'Date': 'first'  # Keep a date for proper sorting
}).reset_index()

fig2 = go.Figure()

fig2.add_trace(go.Scatter(
    x=monthly_loss['Date'],
    y=monthly_loss['Dollar Loss Clean'],
    fill='tozeroy',
    name='Dollar Loss',
    line=dict(color='rgb(67, 147, 195)', width=2),
    fillcolor='rgba(67, 147, 195, 0.3)'
))

fig2.update_layout(
    title={
        'text': 'Total Dollar Loss Trends Over Time',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Time Period',
    yaxis_title='Total Dollar Loss ($)',
    template='plotly_white',
    showlegend=False,
    plot_bgcolor='white',
    xaxis=dict(tickangle=45),
    yaxis=dict(tickformat='$,.0f')
)

fig2.show()

In [5]:
# 3. Seasonal patterns in fraud reporting (Heatmap)
# Create month-year cross tabulation
seasonal_pattern = pd.pivot_table(
    data=df,
    values='Dollar Loss Clean',
    index='Month',
    columns='Year',
    aggfunc='count'
).fillna(0)

# Create month labels
month_labels = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
seasonal_pattern.index = month_labels

fig3 = px.imshow(
    seasonal_pattern,
    aspect='auto',
    color_continuous_scale='Viridis',
    title='Seasonal Patterns in Fraud Reporting'
)

fig3.update_layout(
    title={
        'text': 'Seasonal Patterns in Fraud Reporting',
        'x': 0.5,
        'xanchor': 'center'
    },
    xaxis_title='Year',
    yaxis_title='Month',
    template='plotly_white',
    plot_bgcolor='white',
    height=500
)

# Update color bar title
fig3.update_traces(colorbar_title='Number of Cases')

fig3.show()